# Visão Computacional: Representação de Imagens e Convoluções

Neste notebook, exploraremos os fundamentos da Visão Computacional, focando em como as imagens são representadas computacionalmente e como a operação de convolução é utilizada para extrair características por meio de filtros (kernels).

### Conteúdo Abordado

1.  **Representação de Imagens**:
    * Imagens como tensores (arrays NumPy).
    * Criação e manipulação de imagens em escala de cinza e RGB.
    * Carregamento de imagens externas.
2.  **A Operação de Convolução 2D**:
    * Definição matemática da convolução discreta.
    * Implementação da convolução a partir do zero.
3.  **Filtros (Kernels) e suas Aplicações**:
    * Filtro de suavização (Blur).
    * Filtro de realce de detalhes (Sharpen).
    * Filtros de detecção de bordas (Sobel).

## 1. Representação de Imagens com NumPy

Uma imagem digital é, fundamentalmente, uma estrutura de dados numérica. Para o computador, uma imagem é um tensor (ou um array multidimensional) de valores de intensidade de pixel. Em uma imagem em escala de cinza (*grayscale*), temos uma matriz 2D onde cada elemento representa a intensidade de um pixel, geralmente em um intervalo de [0, 255]. Para imagens coloridas, o modelo mais comum é o RGB, que utiliza um tensor 3D, onde a terceira dimensão representa os canais de cor: Vermelho (Red), Verde (Green) e Azul (Blue).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define as dimensões da imagem
height, width = 100, 100

# Cria uma imagem preta (todos os pixels com valor 0)
grayscale_image = np.zeros((height, width), dtype=np.uint8)

print(f"Shape da imagem em escala de cinza: {grayscale_image.shape}")

# Vamos adicionar um retângulo branco no centro
# O valor 255 corresponde à cor branca
start_row, end_row = height // 4, 3 * height // 4
start_col, end_col = width // 4, 3 * width // 4
grayscale_image[start_row:end_row, start_col:end_col] = 255

# Exibe a imagem
plt.imshow(grayscale_image, cmap='gray', vmin=0, vmax=255)
plt.title("Imagem Grayscale Gerada com NumPy")
plt.show()

### Indexação e Modificação de Imagens

Como as imagens são representadas por arrays NumPy, podemos usar as operações de indexação e *slicing* para acessar e modificar regiões específicas da imagem.

In [ ]:
# Faz uma cópia da imagem anterior para modificação
modified_image = grayscale_image.copy()

# Altera um quadrante da imagem para um tom de cinza médio (valor 128)
modified_image[0:height//2, 0:width//2] = 128

# Exibe a imagem modificada
plt.imshow(modified_image, cmap='gray', vmin=0, vmax=255)
plt.title("Imagem Modificada")
plt.show()

### Imagens Coloridas (RGB)

Imagens coloridas no padrão RGB são representadas como tensores 3D com shape `(altura, largura, 3)`. A terceira dimensão, de tamanho 3, corresponde aos canais de Vermelho, Verde e Azul. Cada pixel `(i, j)` é, portanto, um vetor de três valores `[R, G, B]`.

In [ ]:
# Define as dimensões
height, width = 100, 100

# Cria uma imagem preta
rgb_image = np.zeros((height, width, 3), dtype=np.uint8)

print(f"Shape da imagem RGB: {rgb_image.shape}")

# Adiciona uma faixa vermelha na imagem
# O canal 0 é o Vermelho (Red)
rgb_image[:, 0:width//3, 0] = 255

# Adiciona uma faixa verde na imagem
# O canal 1 é o Verde (Green)
rgb_image[:, width//3:2*width//3, 1] = 255

# Adiciona uma faixa azul na imagem
# O canal 2 é o Azul (Blue)
rgb_image[:, 2*width//3:width, 2] = 255

# Exibe a imagem
plt.imshow(rgb_image)
plt.title("Imagem RGB Gerada com NumPy")
plt.show()

In [ ]:
def draw_circle(image, center, radius, value=255):
    cy, cx = center
    y = np.arange(image.shape[0])[:, None]  # coluna
    x = np.arange(image.shape[1])[None, :]  # linha
    mask = (x - cx) ** 2 + (y - cy) ** 2 <= radius ** 2
    image[mask] = value

# Define dimensões da imagem
height, width = 300, 400

# Cria canais R, G e B com zeros
red_channel = np.zeros((height, width), dtype=np.uint8)
green_channel = np.zeros((height, width), dtype=np.uint8)
blue_channel = np.zeros((height, width), dtype=np.uint8)

# Parâmetros dos círculos
radius = 80
center_offset = 60

center_red = (height // 2 - center_offset // 2, width // 2 - center_offset)
center_green = (height // 2 - center_offset // 2, width // 2 + center_offset)
center_blue = (height // 2 + int(center_offset * 0.8), width // 2)

# Desenha cada círculo no seu canal
draw_circle(red_channel, center_red, radius)
draw_circle(green_channel, center_green, radius)
draw_circle(blue_channel, center_blue, radius)

# Junta os canais em imagem RGB
color_circles_image = np.stack([red_channel, green_channel, blue_channel], axis=-1)

# Exibe o resultado
plt.figure(figsize=(10, 8))
plt.imshow(color_circles_image)
plt.title("Círculos nos Canais R, G e B")
plt.axis('off')
plt.show()

### Carregando Imagens Reais

Na prática, raramente criamos imagens do zero. O fluxo de trabalho comum envolve carregar imagens de arquivos. Bibliotecas como `Pillow` (PIL) e `OpenCV` são usadas para essa finalidade. `Matplotlib` também pode carregar e exibir imagens.

A biblioteca **OpenCV** (`cv2`) é o padrão de fato para a maioria das tarefas de Visão Computacional. Ela é altamente otimizada, implementada em C/C++ com wrappers para Python, e oferece uma vasta gama de funcionalidades prontas para processamento de imagem e vídeo.

A função principal para carregar uma imagem é `cv2.imread()`. É importante notar duas características:
1.  **Formato de Dados**: O OpenCV carrega imagens diretamente como arrays NumPy, o que facilita a integração com o ecossistema de computação científica do Python.
2.  **Ordem dos Canais de Cor**: Por razões históricas, o OpenCV carrega imagens coloridas no formato **BGR** (Blue, Green, Red), e não no tradicional RGB. Bibliotecas de visualização como o Matplotlib esperam imagens em RGB. Portanto, ao carregar uma imagem colorida com `cv2` para exibi-la com `plt`, é quase sempre necessário converter o espaço de cores de BGR para RGB.

A função `cv2.imread()` aceita um segundo argumento, uma *flag* que define como a imagem deve ser carregada:
* `cv2.IMREAD_COLOR` ou `1`: Carrega a imagem em BGR (padrão).
* `cv2.IMREAD_GRAYSCALE` ou `0`: Converte a imagem para escala de cinza.
* `cv2.IMREAD_UNCHANGED` ou `-1`: Carrega a imagem como está, incluindo o canal alfa (transparência), se houver.

In [ ]:
import requests

# URL da imagem de exemplo
image_url = "https://lirp.cdn-website.com/7ece8951/dms3rep/multi/opt/972752326-640w.jpg"
image_filename = "dog.jpg"

# Baixa a imagem
response = requests.get(image_url)
with open(image_filename, "wb") as f:
    f.write(response.content)

In [ ]:
import cv2

# Carrega a imagem em escala de cinza usando OpenCV
gray_image_loaded = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)

print(f"Shape da imagem grayscale (OpenCV): {gray_image_loaded.shape}")
print(f"Tipo de dados (OpenCV): {gray_image_loaded.dtype}")

plt.imshow(gray_image_loaded, cmap='gray')
plt.title("Imagem Grayscale Carregada com OpenCV")
plt.show()

### A Conversão de BGR para RGB

Agora, vamos carregar a mesma imagem em modo colorido para observar a inversão de canais. Ao exibi-la diretamente com `matplotlib`, as cores parecerão incorretas. A correção é feita com a função `cv2.cvtColor()`, especificando a conversão `cv2.COLOR_BGR2RGB`.

In [ ]:
# Carrega a imagem colorida (padrão BGR)
bgr_image_loaded = cv2.imread(image_filename, cv2.IMREAD_COLOR)

# Converte de BGR para RGB para a visualização correta
rgb_image_loaded = cv2.cvtColor(bgr_image_loaded, cv2.COLOR_BGR2RGB)

print(f"Shape da imagem colorida: {bgr_image_loaded.shape}")

# Exibe as imagens para comparação
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(bgr_image_loaded)
axes[0].set_title("Imagem BGR (Visualização Incorreta)")

axes[1].imshow(rgb_image_loaded)
axes[1].set_title("Imagem RGB (Após Conversão)")

plt.show()

### Manipulação de Canais de Cor

Como vimos, uma imagem RGB é um tensor de shape `(altura, largura, 3)`. Essa terceira dimensão contém os canais de cor Vermelho, Verde e Azul. Podemos pensar nela como três matrizes 2D (imagens em escala de cinza) empilhadas, onde cada matriz representa a intensidade de uma cor primária em cada pixel.

Ao isolar esses canais, podemos analisá-los e modificá-los individualmente. Usando o fatiamento (slicing) do NumPy, podemos acessar cada canal facilmente.

In [ ]:
# Separa os canais de cor da imagem RGB
# Lembre-se: 0 = Red, 1 = Green, 2 = Blue
red_channel = rgb_image_loaded[:, :, 0]
green_channel = rgb_image_loaded[:, :, 1]
blue_channel = rgb_image_loaded[:, :, 2]

# Para visualizar cada canal, criamos uma imagem colorida onde os outros canais são zerados.
# Isso mostra a contribuição de cada cor primária para a imagem final.
zeros = np.zeros_like(red_channel) # Matriz de zeros com as mesmas dimensões de um canal

# Cria uma imagem contendo apenas o canal vermelho
image_only_red = np.stack([red_channel, zeros, zeros], axis=2)

# Cria uma imagem contendo apenas o canal verde
image_only_green = np.stack([zeros, green_channel, zeros], axis=2)

# Cria uma imagem contendo apenas o canal azul
image_only_blue = np.stack([zeros, zeros, blue_channel], axis=2)


# Exibe os canais separados
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
axes[0].imshow(rgb_image_loaded)
axes[0].set_title("Imagem Original RGB")
axes[1].imshow(image_only_red)
axes[1].set_title("Canal Vermelho (Red)")
axes[2].imshow(image_only_green)
axes[2].set_title("Canal Verde (Green)")
axes[3].imshow(image_only_blue)
axes[3].set_title("Canal Azul (Blue)")

for ax in axes:
    ax.axis('off')

plt.show()

### Modificando e Unindo Canais

A verdadeira flexibilidade surge quando modificamos os canais de forma independente e depois os unimos novamente para criar uma nova imagem. Operações como alterar o brilho de um canal, aplicar filtros seletivamente ou adicionar elementos gráficos a apenas um canal são comuns em processamento de imagem.

Após a manipulação, podemos usar a função `cv2.merge()` ou `np.stack()` para recombinar as matrizes 2D em um único tensor 3D, formando a imagem colorida final.

In [ ]:
# Vamos fazer uma cópia dos canais para não alterar os originais
red_mod = red_channel.copy()
green_mod = green_channel.copy()
blue_mod = blue_channel.copy()

# Modificação 1: Reduzir a intensidade do canal verde em 50%
# Isso dará à imagem um tom mais roxo/magenta
green_mod = (green_mod * 0.5).astype(np.uint8)

# Modificação 2: Adicionar uma sobreposição (overlay) apenas no canal azul
# Vamos adicionar um círculo branco no centro do canal azul.
h, w = blue_mod.shape
center_x, center_y = w // 2, h // 2
radius = min(h, w) // 4
cv2.circle(blue_mod, (center_x, center_y), radius, 255, -1) # 255=branco, -1=preenchido

# Une os canais modificados para formar uma nova imagem
# A ordem deve ser [R, G, B]
merged_image = cv2.merge([red_mod, green_mod, blue_mod])


# Exibe a imagem original e a modificada
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(rgb_image_loaded)
axes[0].set_title("Imagem Original")
axes[1].imshow(merged_image)
axes[1].set_title("Imagem com Canais Modificados")

for ax in axes:
    ax.axis('off')

plt.show()

## 2. A Operação de Convolução 2D

A convolução é uma operação matemática fundamental em processamento de imagens e a base das Redes Neurais Convolucionais (CNNs). Ela consiste em aplicar um filtro (ou kernel), que é uma pequena matriz de pesos, sobre a imagem de entrada. O kernel desliza sobre a imagem, e para cada posição, é calculada a soma ponderada dos pixels da vizinhança, onde os pesos são os próprios valores do kernel.

A convolução discreta 2D é definida como:

$$(I * K)(i, j) = \sum_{m} \sum_{n} I(i-m, j-n) K(m, n)$$

Onde $I$ é a imagem de entrada e $K$ é o kernel. O resultado é um novo mapa de características (*feature map*), que representa a ativação do filtro em cada posição da imagem.

### Implementando a Convolução

Para entender o processo, vamos implementar uma função de convolução 2D. Nossa implementação utilizará um *padding* do tipo 'valid', o que significa que não adicionaremos preenchimento nas bordas da imagem. Como resultado, a imagem de saída será menor que a de entrada.

In [ ]:
def convolution2d(image, kernel):
    # Dimensões da imagem e do kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # Dimensões da imagem de saída
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1

    # Inicializa a matriz de saída com zeros
    output = np.zeros((output_height, output_width))

    # Itera sobre cada pixel da imagem para aplicar o kernel
    for y in range(output_height):
        for x in range(output_width):
            # Extrai a região da imagem correspondente ao kernel
            region = image[y:y + kernel_height, x:x + kernel_width]
            
            # Aplica a operação de convolução (produto escalar elemento a elemento e soma)
            output[y, x] = np.sum(region * kernel)

    return output

## 3. Aplicação de Filtros (Kernels)

Diferentes kernels podem ser usados para extrair diferentes tipos de características de uma imagem. Vamos explorar alguns dos filtros mais comuns.

### Filtro de Suavização (Blur)

Um filtro de suavização, também conhecido como filtro de média ou *blur*, tem como objetivo principal atenuar ruídos e detalhes de alta frequência em uma imagem. A intuição é simples: cada pixel da imagem de saída é o resultado da média dos valores dos pixels em sua vizinhança na imagem de entrada. Esse processo "espalha" as intensidades dos pixels, resultando em transições mais suaves e uma aparência geral mais "borrada".

O kernel para um *blur* 3x3 é uma matriz onde todos os elementos são iguais. Para preservar o brilho geral da imagem, a soma de todos os elementos do kernel deve ser 1. Portanto, para um kernel de dimensão $3 \times 3$, cada elemento terá o valor de $1/9$.

$$
K_{blur} = \frac{1}{9} \begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

Ao aplicar este kernel, o valor do pixel central é efetivamente substituído pela média de si mesmo e de seus oito vizinhos imediatos.

In [ ]:
# Kernel de Blur 3x3
# Cada pixel será substituído pela média de seus vizinhos
blur_kernel = np.array([
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9]
])

# Aplica o filtro de blur na imagem
blurred_image = convolution2d(gray_image_loaded, blur_kernel)

# Exibe os resultados
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gray_image_loaded, cmap='gray')
axes[0].set_title("Imagem Original")
axes[1].imshow(blurred_image, cmap='gray')
axes[1].set_title("Imagem com Filtro de Blur")
plt.show()

### Filtro de Realce (Sharpen)

O filtro de *sharpening* (nitidez ou realce) opera de forma oposta ao de suavização. Seu objetivo é realçar as bordas e os detalhes finos, aumentando o contraste entre os pixels adjacentes. A ideia por trás deste filtro é acentuar a diferença entre um pixel e a média de sua vizinhança.

Um kernel de *sharpen* comum consegue isso atribuindo um peso positivo alto ao pixel central e pesos negativos aos seus vizinhos.

$$
K_{sharpen} = \begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}
$$

Analisando este kernel:
* O valor central (`5`) multiplica a intensidade do pixel original.
* Os valores vizinhos (`-1`) subtraem a intensidade dos pixels ao redor.
* Se uma região é "plana" (todos os pixels têm valores semelhantes), a soma ponderada resultará em um valor próximo ao original, pois a soma dos pesos do kernel é $5 - 1 - 1 - 1 - 1 = 1$. Isso garante que áreas sem bordas não sejam alteradas drasticamente.
* Em uma borda, onde o valor do pixel central difere significativamente de seus vizinhos, a operação amplificará essa diferença, tornando a borda mais nítida.

In [ ]:
# Kernel de Sharpen
sharpen_kernel = np.array([
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]
])

# Aplica o filtro de sharpen
sharpened_image = convolution2d(gray_image_loaded, sharpen_kernel)

# Exibe os resultados
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gray_image_loaded, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Imagem Original")
axes[1].imshow(sharpened_image, cmap='gray', vmin=0, vmax=255)
axes[1].set_title("Imagem com Filtro Sharpen")
plt.show()

### Filtros de Detecção de Bordas (Sobel)

A detecção de bordas é uma tarefa importante em visão computacional. O operador de Sobel é um dos algoritmos clássicos para isso. Ele utiliza dois kernels, um para detectar bordas horizontais ($K_x$) e outro para bordas verticais ($K_y$).

$$
K_x = \begin{bmatrix}
-1 & 0 & +1 \\
-2 & 0 & +2 \\
-1 & 0 & +1
\end{bmatrix}
\quad
K_y = \begin{bmatrix}
-1 & -2 & -1 \\
 0 &  0 &  0 \\
+1 & +2 & +1
\end{bmatrix}
$$

Aplicamos ambos os kernels à imagem para obter os gradientes $G_x$ e $G_y$. A magnitude do gradiente, $G = \sqrt{G_x^2 + G_y^2}$, nos dá a intensidade da borda.

In [ ]:
# Kernels de Sobel
sobel_x_kernel = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])

sobel_y_kernel = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
])

# Aplica os filtros para obter os gradientes
gradient_x = convolution2d(gray_image_loaded, sobel_x_kernel)
gradient_y = convolution2d(gray_image_loaded, sobel_y_kernel)

# Exibe os gradientes horizontal e vertical
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gradient_x, cmap='gray')
axes[0].set_title("Gradiente Horizontal (Gx - Bordas Verticais)")
axes[1].imshow(gradient_y, cmap='gray')
axes[1].set_title("Gradiente Vertical (Gy - Bordas Horizontais)")
plt.show()

In [ ]:
# Calcula a magnitude do gradiente
# Para evitar que a imagem de saída seja menor, vamos aplicar o filtro em uma imagem já cortada
h, w = gradient_x.shape
gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)

# Normaliza a imagem para o intervalo [0, 255] para melhor visualização
gradient_magnitude = (gradient_magnitude / np.max(gradient_magnitude)) * 255

# Exibe o resultado final
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(gray_image_loaded, cmap='gray')
axes[0].set_title("Imagem Original")
axes[1].imshow(gradient_magnitude, cmap='gray')
axes[1].set_title("Detecção de Bordas (Magnitude de Sobel)")
plt.show()

### Visualizando a Direção dos Gradientes

Os gradientes $G_x$ e $G_y$ não nos dão apenas a informação sobre a presença de uma borda, mas também a sua **orientação**. Em cada pixel, o par $(G_x, G_y)$ forma um vetor que aponta na direção da maior variação de intensidade (do escuro para o claro). A magnitude desse vetor, que calculamos em seguida, nos dá a "força" da borda, enquanto sua direção nos informa sobre a orientação da borda.

Uma excelente forma de visualizar um campo de vetores é através de um *quiver plot*. Este tipo de gráfico desenha setas sobre uma grade para representar a direção e a magnitude dos vetores em cada ponto.

No código a seguir, faremos o seguinte:
1.  **Subamostragem (Subsampling)**: Desenhar uma seta para cada pixel da imagem resultaria em um gráfico excessivamente denso e ilegível. Por isso, selecionamos apenas um subconjunto de pixels (a cada `step` pixels) para visualizar os vetores de gradiente.
2.  **Desenho das Setas**: Para cada ponto selecionado na grade, desenhamos uma seta (vetor) onde os componentes `(U, V)` correspondem aos valores de `(Gx, Gy)` naquele ponto. A direção da seta indica a direção do gradiente.

In [ ]:
# A imagem de gradiente (Gx, Gy) nos dá a magnitude e direção da mudança de intensidade.
# Podemos visualizar isso usando um quiver plot.
# Para evitar um plot muito denso, vamos amostrar os gradientes
step = 5  # Amostra a cada 'step' pixels
y_coords, x_coords = np.mgrid[0:h:step, 0:w:step]
U = gradient_x[0:h:step, 0:w:step]
V = gradient_y[0:h:step, 0:w:step]

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(gray_image_loaded, cmap='gray') # Exibe a imagem subamostrada
ax.quiver(x_coords, y_coords, U, V, color='red', scale=np.max(np.sqrt(U**2 + V**2)) * 20)
ax.set_title("Direção dos Gradientes (Quiver Plot)")
ax.axis('off') # Remove os eixos para uma visualização mais limpa
plt.show()